In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os

# Load Dataset
c101_file = "VRPTW/c101.csv"

# Create folder if not exists
def create_folder():
    if not os.path.exists('VRPTW'):
        os.makedirs('VRPTW')

# Convert c101.txt to CSV if CSV does not exist
def convert_txt_to_csv():
    txt_file = "VRPTW/c101.txt"
    if not os.path.exists(c101_file) and os.path.exists(txt_file):
        with open(txt_file, 'r') as file:
            lines = file.readlines()

        data = []
        for line in lines[9:]:
            parts = line.split()
            if len(parts) == 7:
                data.append(parts)

        df = pd.DataFrame(data, columns=["CUST_NO", "XCOORD", "YCOORD", "DEMAND", "READY_TIME", "DUE_DATE", "SERVICE_TIME"])
        df = df.apply(pd.to_numeric, errors='coerce')
        df.to_csv(c101_file, index=False)

# Load data directly from CSV file
create_folder()
convert_txt_to_csv()
df = pd.read_csv(c101_file)

# Problem Formulation
class VRPTW:
    def __init__(self, df, vehicle_capacity=200):
        self.customers = df.to_dict('records')
        self.vehicle_capacity = vehicle_capacity
        self.depot = self.customers[0]
        self.customers = self.customers[1:]  # All customers except the depot
        self.num_customers = len(self.customers)
        self.dist_matrix = self.distance_matrix()

    def distance_matrix(self):
        coords = np.array([(c['XCOORD'], c['YCOORD']) for c in [self.depot] + self.customers])
        dist_matrix = np.zeros((len(coords), len(coords)))
        for i in range(len(coords)):
            for j in range(len(coords)):
                dist_matrix[i][j] = np.sqrt((coords[i][0] - coords[j][0])**2 + (coords[i][1] - coords[j][1])**2)
        return dist_matrix

    def is_valid_route(self, route):
        time = 0
        load = 0
        for idx in route:
            cust = self.customers[idx - 1] if idx != 0 else self.depot
            load += cust['DEMAND']
            time = max(time, cust['READY_TIME']) + cust['SERVICE_TIME']
            if time > cust['DUE_DATE'] or load > self.vehicle_capacity:
                return False
        return True

    def objective(self, route):
        total_distance = sum(self.dist_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))
        return total_distance

# Implement ACO for VRPTW
class AntColonyOptimization:
    def __init__(self, vrptw, num_ants=25, iterations=100, alpha=1, beta=2, evaporation_rate=0.2):
        self.vrptw = vrptw
        self.num_ants = num_ants
        self.iterations = iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.pheromone = np.ones((self.vrptw.num_customers + 1, self.vrptw.num_customers + 1)) * 0.5  # Higher initial value

    def run(self):
        best_routes = []
        best_distance = float('inf')
        for iteration in range(self.iterations):
            routes = []
            distances = []
            for ant in range(self.num_ants):
                route = self.construct_solution()
                split_routes = self.split_routes(route)
                total_distance = sum(self.vrptw.objective(r) for r in split_routes)
                routes.append(split_routes)
                distances.append(total_distance)
                if total_distance < best_distance:
                    best_routes = split_routes
                    best_distance = total_distance
            self.update_pheromones(routes, distances)
        return best_routes, best_distance

    def construct_solution(self):
        unvisited = list(range(1, self.vrptw.num_customers + 1))
        route = [0]
        while unvisited:
            current = route[-1]
            probabilities = []
            for j in unvisited:
                prob = (self.pheromone[current][j] ** self.alpha) * ((1 / self.vrptw.dist_matrix[current][j]) ** self.beta)
                probabilities.append(prob)
            probabilities = np.array(probabilities) / np.sum(probabilities)

            # Introduce some randomness to exploration
            if random.random() < 0.05:  # 5% chance for random selection
                next_customer = random.choice(unvisited)
            else:
                next_customer = random.choices(unvisited, probabilities)[0]

            route.append(next_customer)
            unvisited.remove(next_customer)
        route.append(0)  # Return to depot
        return route

    def split_routes(self, route):
        split_routes = []
        current_route = [0]
        current_load = 0
        current_time = 0

        for idx in route[1:]:
            if idx == 0:
                continue
            customer = self.vrptw.customers[idx - 1]
            new_load = current_load + customer['DEMAND']
            travel_time = self.vrptw.dist_matrix[current_route[-1]][idx]
            new_time = max(current_time + travel_time, customer['READY_TIME']) + customer['SERVICE_TIME']

            # Allow more flexibility by ensuring maximum usage of capacity and time windows
            if (new_load > self.vrptw.vehicle_capacity * 1.1) or (new_time > customer['DUE_DATE'] * 1.1):
                current_route.append(0)
                split_routes.append(current_route)
                current_route = [0]
                current_load = 0
                current_time = 0

            current_route.append(idx)
            current_load += customer['DEMAND']
            current_time = new_time

        current_route.append(0)
        split_routes.append(current_route)
        return split_routes

    def update_pheromones(self, routes, distances):
        self.pheromone *= (1 - self.evaporation_rate)
        global_best_distance = min(distances)
        global_best_route = routes[distances.index(global_best_distance)]

        # Reinforce the pheromone for the global best route
        for route in global_best_route:
            for i in range(len(route) - 1):
                self.pheromone[route[i]][route[i + 1]] += 1 / global_best_distance

        # Update pheromones for all routes, reducing influence from inefficient routes
        for route_set, distance in zip(routes, distances):
            if distance == global_best_distance:
                continue  # Skip global best, already updated
            for route in route_set:
                for i in range(len(route) - 1):
                    self.pheromone[route[i]][route[i + 1]] += 0.1 / distance  # Less weight for suboptimal routes

# Run ACO on VRPTW
def evaluate_aco(vrptw):
    aco = AntColonyOptimization(vrptw, alpha=1, beta=2, evaporation_rate=0.2)
    best_routes, best_distance = aco.run()
    for idx, route in enumerate(best_routes):
        if len(route) > 2:  # Only print routes with customers
            route_str = " -> ".join(map(str, route[1:-1]))
            print(f"Route {idx + 1}: {route_str} | Distance: {vrptw.objective(route):.2f}")
    print(f"Total Distance: {best_distance:.2f}")

# Run Evaluation
if __name__ == "__main__":
    create_folder()
    convert_txt_to_csv()
    vrptw = VRPTW(df)
    evaluate_aco(vrptw)


Route 1: 70 -> 73 -> 77 -> 79 -> 80 -> 69 -> 47 | Distance: 146.17
Route 2: 46 | Distance: 41.18
Route 3: 45 | Distance: 44.72
Route 4: 48 | Distance: 46.65
Route 5: 59 | Distance: 70.11
Route 6: 66 | Distance: 33.11
Route 7: 1 | Distance: 37.36
Route 8: 75 | Distance: 31.62
Route 9: 8 | Distance: 36.22
Route 10: 6 | Distance: 38.00
Route 11: 9 | Distance: 40.20
Route 12: 10 | Distance: 33.53
Route 13: 11 | Distance: 39.29
Route 14: 51 | Distance: 50.00
Route 15: 50 | Distance: 45.61
Route 16: 52 | Distance: 42.43
Route 17: 49 | Distance: 38.42
Route 18: 26 | Distance: 31.62
Route 19: 23 | Distance: 26.00
Route 20: 22 | Distance: 24.33
Route 21: 21 | Distance: 20.40
Route 22: 17 | Distance: 66.60
Route 23: 19 | Distance: 78.10
Route 24: 16 | Distance: 80.62
Route 25: 14 | Distance: 78.71
Route 26: 12 | Distance: 76.16
Route 27: 87 | Distance: 50.99
Route 28: 91 | Distance: 44.72
Route 29: 88 | Distance: 53.85
Route 30: 89 | Distance: 48.70
Route 31: 90 | Distance: 41.23
Route 32: 82 | 